# Title

In [34]:
## Load libraries

In [35]:
import pandas as pd
import requests
import gzip
import json
import io
import numpy as np
import re

## Data Collection and Preprocessing

### Importing Data

First, we import in the data for google reviews in Vermont, USA, and convert it into a dataframe entitled `vt`.

In [36]:
url = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Vermont_10.json.gz'

response = requests.get(url, stream = True)
response.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response.content), mode = 'rb') as gz_file:
    data_list = [json.loads(line) for line in gz_file]

vt = pd.DataFrame(data_list)

In [37]:
print(vt.shape) # 324725 observations and 8 columns.
print(vt.columns)

(324725, 8)
Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')


Next, we import in the data for local businesses metadata in Vermont, USA, and convert it into a dataframe entitled `vt_metadata`.

In [38]:
url_metadata = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Vermont.json.gz'

response_metadata = requests.get(url_metadata, stream = True)
response_metadata.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response_metadata.content), mode = 'rb') as gz_file:
    data_list1 = [json.loads(line) for line in gz_file]

vt_metadata = pd.DataFrame(data_list1)

In [39]:
print(vt_metadata.shape) # 11291 observations and 15 columns
print(vt_metadata.columns)

(11291, 15)
Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')


### Data Cleaning
Next, we clean and prepare  the two datasets for analysis. We start with the reviews dataset before moving on to the business metadata dataset.

#### Reviews dataset

We'll perform the following steps on the reviews dataset:

1. Drop the irrelevant `name` column to avoid redundancy as users can be identified by `user_id`
2. Convert text in all columns to lower case
3. Clean `text` by replacing multiple spaces with a single space
4. Remove duplicate reviews based on user id, review text comment, business location, and time uploaded, to ensure each review is unique

In [40]:
vt = vt.drop('name', axis = 1) # drop 'name' column as it's not relevant to our analysis
vt.columns = vt.columns.str.lower() # convert all strings to lower case
vt['text'] = vt['text'].str.replace(r'\s+', ' ', regex = True) # clean text 
vt = vt.drop_duplicates(subset = ['user_id', 'text', 'gmap_id', 'time']) # drop duplicate 

Next, as the urls for the pictures in the `pics` column is presented in a nested dictionary, we write a function to collapse all urls into a single python list. We name the new column as `pics_collapsed` and drop the old `pics` column.

In [41]:
def collapse_pics(pic_list):
    if not pic_list:
        return []  
    urls = []
    for pic_dict in pic_list:
        urls.extend(pic_dict.get('url', []))
    return urls

vt['pics_collapsed'] = vt['pics'].apply(collapse_pics)
vt = vt.drop('pics', axis = 1)

Similarly, we write a function to collapse business responses to reviews in the `resp` column, naming the new column as `resp_collapsed` and dropping the old `resp` column.

In [42]:
def extract_texts(resp_entry):
    if isinstance(resp_entry, dict):
        # single response dict
        return resp_entry.get("text", "")
    elif isinstance(resp_entry, list):
        # list of response dicts
        return " ".join([d.get("text", "") for d in resp_entry if isinstance(d, dict)])
    elif isinstance(resp_entry, str):
        # fallback: extract with regex if it's a string
        texts = re.findall(r'"text":\s*"([^"]*)"', resp_entry)
        return " ".join(texts)
    else:
        return ""
    
vt["resp_collapsed"] = vt["resp"].apply(extract_texts)
vt = vt.drop("resp", axis = 1)

The first 3 observations of the reviews dataframe are as such.

In [43]:
print(vt.head(3))

                 user_id           time  rating  \
0  118026874392842649478  1620085852324       5   
1  101532740754036204131  1580309946474       5   
2  115404122636203550540  1605195974445       5   

                                                text  \
0      Always done right from wood stove to screens!   
1  A great company to work with. Their sales and ...   
2  Great place to do business with staff was grea...   

                                 gmap_id pics_collapsed  \
0  0x89e02445cb9db457:0x37f42bff4edf7a43             []   
1  0x89e02445cb9db457:0x37f42bff4edf7a43             []   
2  0x89e02445cb9db457:0x37f42bff4edf7a43             []   

                                      resp_collapsed  
0  Good Evening, Rebecca! Thanks SO much for the ...  
1  Good Afternoon, Peter - Really appreciate the ...  
2  Hi Chad!\n\nThank you so much for the 5-Star r...  


#### Metadata dataset

We'll perform the following steps on the reviews dataset:

1. Drop the irrelevant columns that will not be needed for analysis
2. Convert text in all columns to lower case
3. Clean `description` and `category` columns by replacing multiple spaces with a single space
4. Remove duplicate reviews based on business name and Google Maps id to ensure each observation is unique

In [44]:
vt_metadata = vt_metadata.drop(['address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results'], axis = 1)
vt_metadata.columns = vt_metadata.columns.str.lower() 
vt_metadata['description'] = vt_metadata['description'].str.replace(r'\s+', ' ', regex = True)
vt_metadata['category'] = vt_metadata['category'].str.replace(r'\s+', ' ', regex = True)
vt_metadata = vt_metadata.drop_duplicates(subset = ['name','gmap_id'])

The first 3 observations of the metadata dataframe are as such.

In [45]:
print(vt_metadata.head(3))

                       name                                gmap_id  \
0               Royal Group  0x89e02445cb9db457:0x37f42bff4edf7a43   
1  Foxglove Farm and Forest  0x4cb549e8877cf0d7:0xe8f003e6d73392ae   
2              Carr's Gifts  0x4cb54a301f3518f7:0x39af4eda1efb9117   

  description  category                                                url  
0        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  
1        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  
2        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  


In [46]:
vt_metadata.to_csv('vt_metadata.csv')
vt.to_csv('vt.csv')

### Data Scraping for Business Information

Scraping Process: We used the code below to derive a list of URLs that can be used by [Google Maps Scraper](https://console.apify.com/actors/nwua9Gu5YrADL7ZDj/input) to scrape data from Google Maps. Among the data collected are the `description` and `category` data. However, it has soon become too expensive to scrape all 11k+ entries using this scraper, so we instead used [Google Maps Review Scraper](https://console.apify.com/actors/nwua9Gu5YrADL7ZDj/input) to scrape `category` data, as well as the image URLs of the locations that can be used to retrieve the image to be used by Genma to generate a description for the place, as a potential substitute to textual descriptions.

In [47]:
# Extracting list of URLS 
import pandas as pd

input_file = "vt_metadata.csv"
output_file = "urls.txt"
df = pd.read_csv(input_file)
urls = df['url'].dropna().tolist()
unique_urls = list(set(urls))

with open(output_file, 'w') as f:
    for url in unique_urls:
        f.write(f"{url}\n")
        

The first 3 URLs in the file are as such:

In [48]:
file = open("urls.txt", "r")
for i in range(3):
    print(file.readline())
file.close()

https://www.google.com/maps/place//data=!4m2!3m1!1s0x4cb4ca87cefddb8b:0xcd5c11ad6acdc3aa?authuser=-1&hl=en&gl=us

https://www.google.com/maps/place//data=!4m2!3m1!1s0x4cb507655eb10f69:0x85eacd66a9b02ac1?authuser=-1&hl=en&gl=us

https://www.google.com/maps/place//data=!4m2!3m1!1s0x89e1caeb931f0ef1:0x987ee54770cc0466?authuser=-1&hl=en&gl



Numerous columns are available after the scraping. However, as only the `category` and `imageUrl` columns are of significance, aside from the URL that uniquely identifies each location, these are the only columns downloaded in the resulting dataset.

The first 3 entries are as such:

In [49]:
vt_scraped = pd.read_csv("imcat.csv")
print(vt_scraped.head(3)) # 11291 observations and 3 columns

                                                 url  \
0  https://www.google.com/maps/place//data=!4m2!3...   
1  https://www.google.com/maps/place/Orwell,+VT+0...   
2  https://www.google.com/maps/place/Morristown,+...   

                                               image  \
0  https://lh3.googleusercontent.com/gps-cs-s/AC9...   
1  https://lh3.googleusercontent.com/gps-cs-s/AC9...   
2  https://lh3.googleusercontent.com/gps-cs-s/AC9...   

                                            category  
0  ['Historical society', 'Gift shop', 'History m...  
1                                          ['River']  
2                                  ['Mountain pass']  


### Adding Scraped Data into Metadata

We add the scraped `category` and `image` data into the metadata dataframe if these fields are empty in the metadata dataframe with the same `url`. Since the metadata dataframe does not have an `image` column, we created it before adding the data in.

In [50]:
# Add image column to vt_metadata if not exists
if 'image' not in vt_metadata.columns:
    vt_metadata['image'] = np.nan

# Ensure consistent column names (lowercase, strip spaces)
vt_metadata.columns = [col.strip().lower() for col in vt_metadata.columns]
vt_scraped.columns = [col.strip().lower() for col in vt_scraped.columns]
 
# We assume both have at least these columns: url, description, categories
for idx, row in vt_scraped.iterrows():
    url = row['url']
    
    # Find matching rows in main file
    matches = vt_metadata['url'] == url
    
    if matches.any():
        # Update categories if main file missing
        if pd.notna(row['category']) and row['category'].strip() != "":
            vt_metadata.loc[matches & (vt_metadata['category'].isna() | (str(vt_metadata['category']).strip() == "")),
                        'category'] = row['category']
        
        # Update image if main file missing
        if 'image' in row and pd.notna(row['image']) and row['image'].strip() != "":
            vt_metadata.loc[matches & (vt_metadata['image'].isna() | (str(vt_metadata['image']).strip() == "")),
                        'image'] = row['image']
        

# Save the updated file
vt_metadata.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")


C:\Users\angel\AppData\Local\Temp\ipykernel_24756\5542819.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Historical society', 'Gift shop', 'History museum', 'Local history museum', 'Museum', 'Tourist attraction']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  vt_metadata.loc[matches & (vt_metadata['category'].isna() | (str(vt_metadata['category']).strip() == "")),
C:\Users\angel\AppData\Local\Temp\ipykernel_24756\5542819.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://lh3.googleusercontent.com/gps-cs-s/AC9h4no6TiFPpCZ6GEJ1Zjq8dmAx6qsjSsZ0UzqOiAfTXC5vNloE62FzpW7rHK5b--M1s2dplp_E2Klvc8ECkTELP-n_UJ6QUmnwDKEEUxibSyTsKOTOR7bb4e8fHeru4cb-uyzB2C-nK7k-4Clr=w408-h506-k-no' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  vt_m

Updated file saved as urls.txt


### Merging Reviews and Metadata

We merge the reviews and metadata dataframes by common key `gmap_id`, save the merged dataframe as `vt_merged`. We drop the now irrevelent `gmap_id` column.

In [51]:
vt_merged = pd.merge(vt, vt_metadata, on = 'gmap_id', how = 'inner')
vt_merged = vt_merged.drop('gmap_id', axis = 1)

We insert new column called `review_id` such that each review has a unique identifier.

In [52]:
vt_merged['review_id'] = range(len(vt_merged)) 
id_column = vt_merged.pop('review_id')
vt_merged.insert(0, 'review_id', id_column)

### Manual Labelling of Data

We add 7 new boolean columns:
1. `is_image_ad` TRUE if image uploaded is labelled as advertisement
2. `is_image_irrelevant` TRUE if image uploaded is labelled as irrelevant
3. `is_text_ad` TRUE if text comment is labelled as advertisement
4. `is_text_irrelevant` TRUE if text comment is labelled as irrelevant
5. `is_text_rant` TRUE if text comment is labelled as rant from non-visitor
6. `is_review_ad` TRUE if either image or text comment is labelled as advertisement 
7. `is_review_irrelevant` TRUE if either image or text comment is labelled as irrelevant 

In [53]:
vt_merged['is_image_ad'] = None
vt_merged['is_image_ad'] = vt_merged['is_image_ad'].astype(bool)

vt_merged['is_image_irrelevant'] = None
vt_merged['is_image_irrelevant'] = vt_merged['is_image_irrelevant'].astype(bool)

vt_merged['is_text_ad'] = None
vt_merged['is_text_ad'] = vt_merged['is_text_ad'].astype(bool)

vt_merged['is_text_irrelevant'] = None
vt_merged['is_text_irrelevant'] = vt_merged['is_text_irrelevant'].astype(bool)

vt_merged['is_text_rant'] = None
vt_merged['is_text_rant'] = vt_merged['is_text_rant'].astype(bool)

vt_merged["is_review_ad"] = vt_merged["is_text_ad"] | vt_merged["is_image_ad"]

vt_merged['is_review_irrelevant'] = vt_merged["is_image_irrelevant"] | vt_merged["is_text_irrelevant"]

Next, we add 2 columns for quality check of the review.
1. `helpfulness`
    - `not_helpful` (review was relevant but did not add information to the reader)
    - `helpful` (review provided some helpful information to make decisions about the visit)
    - `very_helpful` (review gave crucial or new information that can significantly impact visit decisions)

2. `sensibility` TRUE if the numerical star rating corresponds to the sentiments present in text review

In [54]:
categories = ["not_helpful", "helpful", "very_helpful"]
vt_merged["helpfulness"] = pd.Categorical(
    values=[""] * len(vt_merged), 
    categories = categories,
    ordered = True
)

vt_merged['sensibility'] = None
vt_merged['sensibility'] = vt_merged['sensibility'].astype(bool)

The columns of the dataframe are as such.

In [55]:
print(vt_merged.columns)

Index(['review_id', 'user_id', 'time', 'rating', 'text', 'pics_collapsed',
       'resp_collapsed', 'name', 'description', 'category', 'url', 'image',
       'is_image_ad', 'is_image_irrelevant', 'is_text_ad',
       'is_text_irrelevant', 'is_text_rant', 'is_review_ad',
       'is_review_irrelevant', 'helpfulness', 'sensibility'],
      dtype='object')


### Filtering for Reviews Containing Text and/or Pictures

We assume that a numerical star rating alone does not provide sufficient ustification on why a review would violate any of the three policies. Hence, in this project we only focus on looking at the reviews containing text comments and/or pictures. This filters out around 45% of all reviews.

In [56]:
vt_rating_only = vt_merged[
    vt_merged["text"].isna() & (vt_merged["pics_collapsed"] == "[]")
]

vt_merged = vt_merged.drop(vt_rating_only.index) # contains text and/or pic


We save the final dataframe as `vt_merged.csv`.